Import packages.

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

Set url.

In [2]:
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

Get the table.

In [3]:
html=requests.get(url)
soup=BeautifulSoup(html.text, 'html.parser')
table=soup.find('table',{'class':'wikitable sortable'})

Build a dataframe from the table.

In [4]:
columns = [th.text.replace('\n', '') for th in table.find('tr').find_all('th')]
trs = table.find_all('tr')[1:]
rows = list()
for tr in trs:
    rows.append([td.text.replace('\n', '').replace('\xa0', '') for td in tr.find_all('td')])
df=pd.DataFrame(data=rows,columns=columns)

Get rid of the rows with no borough assigned and set neighbourghood to borough if not assigned.

In [5]:
df.drop(df[df.Borough=='Not assigned'].index,inplace=True)
df['Neighbourhood'].replace('Not assigned',df['Borough'],inplace=True)

Combine rows with the same postcode and sort by postcodes.

In [6]:
duf=df['Postcode'].duplicated(keep='first')
duf=list(duf[duf==True].index)
dul=df['Postcode'].duplicated(keep='last')
dul=list(dul[dul==True].index)
for i,j in zip(dul[::-1],duf[::-1]):
    df.loc[i,'Neighbourhood']=str(df.loc[j,'Neighbourhood'])+','+str(df.loc[i,'Neighbourhood'])
df.drop(index=duf,inplace=True)
df.sort_values('Postcode',inplace=True)
df.reset_index(drop=True,inplace=True)

Show the final dataframe.

In [7]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern,Rouge"
1,M1C,Scarborough,"Port Union,Rouge Hill,Highland Creek"
2,M1E,Scarborough,"West Hill,Morningside,Guildwood"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [8]:
df.shape

(103, 3)

 Get geo info from csv<br>
 Geocode has been very consitent to return None for me :(

In [9]:
df_geo=pd.read_csv('https://cocl.us/Geospatial_data')
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merge geo info.

In [10]:
df_new=df.merge(df_geo,left_on='Postcode',right_on='Postal Code',how='left')
df_new.drop('Postal Code',axis=1,inplace=True)
df_new.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern,Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Port Union,Rouge Hill,Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"West Hill,Morningside,Guildwood",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
